# Pytorch Quickstart

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [1]:
import torch

In [2]:
x = torch.rand(5, 3)
print(x)

tensor([[0.7413, 0.7380, 0.1778],
        [0.1242, 0.8303, 0.0933],
        [0.8110, 0.9166, 0.6995],
        [0.5725, 0.6785, 0.7004],
        [0.5880, 0.2916, 0.5887]])


In [3]:
torch.cuda.is_available()

False

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Create Models

In [7]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizing the Model Parameters

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298757  [    0/60000]
loss: 2.289601  [ 6400/60000]
loss: 2.276316  [12800/60000]
loss: 2.274573  [19200/60000]
loss: 2.268358  [25600/60000]
loss: 2.224753  [32000/60000]
loss: 2.239569  [38400/60000]
loss: 2.201534  [44800/60000]
loss: 2.191858  [51200/60000]
loss: 2.171807  [57600/60000]
Test Error: 
 Accuracy: 42.4%, Avg loss: 2.163882 

Epoch 2
-------------------------------
loss: 2.170543  [    0/60000]
loss: 2.158558  [ 6400/60000]
loss: 2.108370  [12800/60000]
loss: 2.123218  [19200/60000]
loss: 2.078524  [25600/60000]
loss: 2.014241  [32000/60000]
loss: 2.044525  [38400/60000]
loss: 1.965140  [44800/60000]
loss: 1.961649  [51200/60000]
loss: 1.898824  [57600/60000]
Test Error: 
 Accuracy: 58.8%, Avg loss: 1.893673 

Epoch 3
-------------------------------
loss: 1.924486  [    0/60000]
loss: 1.888659  [ 6400/60000]
loss: 1.776397  [12800/60000]
loss: 1.817757  [19200/60000]
loss: 1.711230  [25600/60000]
loss: 1.658160  [32000/600

### Model Save & Load

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    print(f'pred: "{pred}"')
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

pred: "tensor([[-0.9666, -1.2041, -0.1476, -1.0406, -0.2823,  0.6578, -0.5154,  1.1432,
          0.9284,  1.4916]])"
Predicted: "Ankle boot", Actual: "Ankle boot"
